In [ ]:
%pip install -U starpoint sentence-transformers

In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
STARPOINT_API_KEY = os.environ["STARPOINT_API_KEY"]

In [ ]:
import json

with open('./monster_text.json') as f:
    monster_texts = json.load(f)

# def embed(texts):
#     embed_resp = requests.post(EMBEDDING_URL, headers=headers, json={"text": texts, "model": "MINI6"})
#     print(embed_resp.content)
#     return embed_resp.json()["results"]

def embed(texts):
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(texts)

documents_to_upload = []
embeddings = embed([monster["text"] for monster in monster_texts])
embeddings = [embedding["embedding"] for embedding in embeddings]

for index, monster in enumerate(monster_texts):
    documents_to_upload.append({
        "embedding": embeddings[index],
        "metadata": monster
    })
print(documents_to_upload[:10])

In [ ]:
import copy
from starpoint.db import Client

COLLECTION_NAME = "noteable-dnd"

documents_to_upload = []

for key, embeddings in embeddings_data.items():
    metadata = monsters[key]
    for text_field, embedding in embeddings.items():
        new_metadata = copy.deepcopy(metadata)
        new_metadata['embedding_source'] = text_field
        documents_to_upload.append({
            'metadata': new_metadata,
            'embedding': embedding.tolist()
        })

client = Client(api_key=STARPOINT_API_KEY)
client.insert(documents=documents_to_upload, collection_name=COLLECTION_NAME)

In [ ]:
import requests

EMBEDDING_URL = "https://scholar.starpoint.ai/api/v1/embed"
headers = {"x-starpoint-key": STARPOINT_API_KEY}

question = "how big is an aerosaur?"
embed_resp = embed([question])

# query_embedding = embed_resp.json()["results"][0]["embedding"]
query_embedding = embed_resp[0]



In [ ]:
from sentence_transformers import SentenceTransformer
# from llama_cpp import Llama
from starpoint.db import Client

COLLECTION_ID = '21643fcc-8620-49db-abb2-8b18116567f4'
starpoint_client = Client(api_key=STARPOINT_API_KEY)
GGML_MODEL_PATH = "./model/model.bin"

# llm = Llama(model_path=GGML_MODEL_PATH)

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def query_starpoint(text):
    query_embedding = model.encode(text).tolist()
    relevant_monsters = starpoint_client.query(
        collection_id=COLLECTION_ID,
        query_embedding=query_embedding,
        sql="SELECT * FROM collection LIMIT 5"
    )
    return relevant_monsters


In [ ]:
resp = query_starpoint("how big are Balors?")
print(resp)